In [2]:
%load_ext autoreload
%autoreload 2

import polars as pl
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches
plt.style.use('ggplot')

import numpy as np
from utils import utils


url_data = "./data"

In [6]:
%reload_ext autoreload

In [3]:
os.chdir('../')

In [5]:
speed_high = 15
speed_low = 2
speed_moderate_down = 6
speed_moderate_up = 8
utils.set_speed(speed_high,speed_low,speed_moderate_down,speed_moderate_up)

print(utils.speed_high)
print(utils.speed_low)
print(utils.speed_moderate_up)
print(utils.speed_moderate_down)

utils.get_speeds()

15
2
8
6
15
2
8
6


# Count number of events

In [8]:
range_cut = 96
path_save = os.path.join(url_data,"dataset_split_events_rare")

years = ["2000","2001","2002","2003","2004","2005","2006","2007","2008","2009",\
         "2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

for year in years:
    try:
        print(year,utils.count_event_per_file(os.path.join(url_data,"dataset_summer_split_13",f"{year}_{range_cut}_13_high_summer.csv"))['events'].sum())
    except:
        pass
    # try:
    #     print(utils.count_event_per_file(os.path.join(url_data,"dataset_split_events",f"{year}_{range_cut}_moderate.csv"))['events'].sum())
    # except:
    #     print("empty csv")
    # try:
    #     print(utils.count_event_per_file(os.path.join(url_data,"dataset_split_events",f"{year}_{range_cut}_low.csv"))['events'].sum())
    # except:
    #     print("empty csv")

2000 2280
2001 6420
2002 2930
2009 4560
2010 7679
2013 6529
2014 9046
2017 5008
2019 5775
2020 2052


# Split summer and winter

In [ ]:
import polars as pl
import os
from datetime import datetime

ini_winder = datetime.strptime("2000-01-01 00:00:00",'%Y-%m-%d %H:%M:%S')
end_winter = datetime.strptime("2000-06-30 23:59:59",'%Y-%m-%d %H:%M:%S')
ini_summer = datetime.strptime("2000-07-01 00:00:00",'%Y-%m-%d %H:%M:%S')
end_summer = datetime.strptime("2000-12-31 23:59:59",'%Y-%m-%d %H:%M:%S')

md_ini_winter = (ini_winder.month, ini_winder.day)
md_end_winter = (end_winter.month, end_winter.day)
md_ini_summer = (ini_summer.month, ini_summer.day)
md_end_summer = (end_summer.month, end_summer.day)

for file in os.listdir("../data/dataset_split_events_15/"):
    df = pl.read_csv("../data/dataset_split_events_15/" + file)

    cut_size = 96

    middle = int(cut_size/2)

    winter_df = pl.DataFrame()
    summer_df = pl.DataFrame()

    for windfarm in range(0,38):
        # print("windfarm nº",windfarm)
        df_new = df.filter(pl.col("index") == windfarm)
        n_events = df_new["n_event"].unique().shape[0]
        # Compute number of rows in summer and in winter for the events in the middle

        count_winter = 0
        count_summer = 0

        for event in range(0,n_events):
            df_new_event = df_new.filter(pl.col("n_event") == event)
            time_split = datetime.strptime(df_new_event[middle,0],'%Y-%m-%d %H:%M:%S')
            date_md = (time_split.month,time_split.day)

            if md_ini_winter <= date_md <= md_end_winter:
                df_new_event = df_new_event.with_column(pl.lit(count_winter).alias("n_event"))
                winter_df = pl.concat([winter_df,df_new_event], rechunk=True)
                count_winter += 1 
            else:
                df_new_event = df_new_event.with_column(pl.lit(count_summer).alias("n_event"))
                summer_df = pl.concat([summer_df,df_new_event], rechunk=True)
                count_summer += 1 

    summer_df.write_csv(os.path.join(f"../data/dataset_summer_split_15/",f"{file[:-4]}summer.csv"))
    winter_df.write_csv(os.path.join(f"../data/dataset_winter_split_15/",f"{file[:-4]}winter.csv"))


# Create list of files by 

In [11]:
import os

path = "./IKM/clustering/"
file_clusters = "cluster_split.txt"

with open(os.path.join(path,file_clusters), 'r') as file:
    # Lee el contenido del archivo
    data = file.readlines()

new_data = []
for line in data:
    new_data.append(line.strip().split('\n')[0])

index_1 = new_data.index('Cl#:0')
index_2 = new_data.index('Cl#:1')
index_3 = new_data.index('Cl#:2')
index_4 = new_data.index('Coefficients')

cluster_1 = new_data[index_1+1 : index_2]
cluster_2 = new_data[index_2+1 : index_3]
cluster_3 = new_data[index_3+1 : index_4]
coeff = new_data[index_4+1 :]



for index,info in enumerate(cluster_1):
    cluster_1[index] = "../../../data/file_per_event/current_experiment/" + info + ".txt"
    # print("\"" + cluster_1[index] + ".txt\",")

print()

for index,info in enumerate(cluster_2):
    cluster_2[index] = "../../../data/file_per_event/current_experiment/" + info + ".txt"
    # print("\"" + cluster_2[index] + ".txt\",")

print()

for index,info in enumerate(cluster_3):
    cluster_3[index] = "../../../data/file_per_event/current_experiment/" + info + ".txt"
    # print("\"" + cluster_3[index] + ".txt\",")

print(cluster_1)

print()
print(coeff)




['../../../data/file_per_event/current_experiment/0_0_2000', '../../../data/file_per_event/current_experiment/0_102_2000', '../../../data/file_per_event/current_experiment/0_103_2000', '../../../data/file_per_event/current_experiment/0_10_2000', '../../../data/file_per_event/current_experiment/0_113_2000', '../../../data/file_per_event/current_experiment/0_11_2000', '../../../data/file_per_event/current_experiment/0_12_2000', '../../../data/file_per_event/current_experiment/0_13_2000', '../../../data/file_per_event/current_experiment/0_14_2000', '../../../data/file_per_event/current_experiment/0_15_2000', '../../../data/file_per_event/current_experiment/0_17_2000', '../../../data/file_per_event/current_experiment/0_19_2000', '../../../data/file_per_event/current_experiment/0_16_2000', '../../../data/file_per_event/current_experiment/0_18_2000']

['0.0 1.002 0.0 0.0 -0.0 0.001 0.0 0.015 0.0 -0.016 0.0 0.0 -0.0', '0.0 1.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0', '0.0 1.0 0.0 -0.0

# Create file for event

In [ ]:
import polars as pl
import numpy as np
import os
from tqdm import tqdm

path = "../data/final_files/small/"
path_save_file_per_event = "../../../data/file_per_event/current_experiment"

def create_files_per_event(event,name,path):

    if not os.path.exists(os.path.join(path,f'{name}.txt')):
        event.write_csv(os.path.join(path,f'{name}.txt'))
        
    return None

for file in os.listdir():

    data = pl.read_csv(os.path.join(path, file))

    categories = np.array(['_'.join(str(item) for item in sublist) for sublist in data[:,1:4].rows()])

    # Get unique categories
    unique_categories = np.unique(categories)
    np.random.shuffle(unique_categories)

    # Split the array into a list of lists based on categories
    for index,category in enumerate(tqdm(unique_categories, desc='Split events, and create TSObject', leave=False)):
        # Select the one event in a big dataframe
        indices = np.where(categories == category)
        subset = data[indices[0]]

        create_files_per_event(subset,category,path_save_file_per_event)
